# TimeSequence - Syft Duet - Data Owner 🎸

Contributed by [@Koukyosyumei](https://github.com/Koukyosyumei)

## PART 1: Launch a Duet Server and Connect

As a Data Owner, you want to allow someone else to perform data science on data that you own and likely want to protect.

In order to do this, we must load our data into a locally running server within this notebook. We call this server a "Duet".

To begin, you must launch Duet and help your Duet "partner" (a Data Scientist) connect to this server.

You do this by running the code below and sending the code snippet containing your unique Server ID to your partner and following the instructions it gives!

In [ ]:
import os
from typing import Any
from typing import List as TypeList

import numpy as np
import torch
import torchvision
import torchvision.utils as vutils
from syft import SyModule
from syft.core.plan.plan_builder import ROOT_CLIENT, make_plan

try:
    # make notebook progress bars nicer
    from tqdm.notebook import tqdm
except ImportError:
    print(f"Unable to import tqdm")

In [ ]:
import syft as sy

duet = sy.launch_duet(loopback=True)
sy.logger.add(sink="./syft_do.log")

In [ ]:
# handler with no tags accepts everything. Better handlers coming soon.
duet.requests.add_handler(action="accept")

In [ ]:
T = 5  # original number is 20
L = 100  # original number is 1000
N = 100

x = np.empty((N, L), "int64")
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype("float64")

In [ ]:
data_tensor = torch.Tensor(data)
data_tensor.tag("data")
data_tensor.send(duet, pointable=True)

In [ ]:
L_ptr = sy.lib.python.Int(L)
L_ptr.tag("L")
L_ptr.send(duet, pointable=True)

In [ ]:
input = data_tensor[3:, :-1]
input.tag("input")
input.send(duet, pointable=True)

target = data_tensor[3:, 1:]
target.tag("target")
target.send(duet, pointable=True)

test_input = data_tensor[:3, :-1]
test_input.tag("test_input")
test_input.send(duet, pointable=True)

test_target = data_tensor[:3, 1:]
test_target.tag("test_target")
test_target.send(duet, pointable=True)

In [ ]:
input_size_0 = sy.lib.python.Int(input.size()[0])
input_size_0.tag("input_size_0")
input_size_0.send(duet, pointable=True)

In [ ]:
remote_torch = ROOT_CLIENT.torch
remote_python = ROOT_CLIENT.python

In [ ]:
class Sequence(SyModule):
    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)
        self.lstm1 = torch.nn.LSTMCell(1, 51)
        self.lstm2 = torch.nn.LSTMCell(51, 51)
        self.linear = torch.nn.Linear(51, 1)
        self.future = 0

    def forward(self, input: Any) -> Any:
        outputs = remote_python.List([])
        h_t = torch.zeros(97, 51)
        c_t = torch.zeros(97, 51)
        h_t2 = torch.zeros(97, 51)
        c_t2 = torch.zeros(97, 51)

        state_1 = remote_python.Tuple((h_t, c_t))
        state_2 = remote_python.Tuple((h_t2, c_t2))

        for input_t in input.split(1, dim=1):
            state_1 = self.lstm1(input_t, state_1)
            state_2 = self.lstm2(h_t, state_2)
            output = self.linear(state_2[0])
            outputs.append(output)

        for i in range(self.future):  # if we should predict the future
            state_1 = self.lstm1(output, state_1)
            state_2 = self.lstm2(h_t, state_2)
            output = self.linear(state_2[0])
            outputs.append(output)

        outputs = remote_torch.cat(outputs, dim=1)
        return outputs

In [ ]:
dummy_size = 9
dummy_input = input[:, :dummy_size]
dummy_target = target[:, :dummy_size]
local_model = Sequence(input_size=(input_size_0, dummy_size))

In [ ]:
@make_plan
def train(
    input: torch.Tensor = dummy_input,
    target: torch.Tensor = dummy_target,
    model: SyModule = local_model,
) -> TypeList:

    # optimizer = remote_torch.optim.LBFGS(model.parameters(), lr=1e-2)
    optimizer = remote_torch.optim.AdamW(model.parameters(), lr=1e-2)
    criterion = remote_torch.nn.MSELoss()

    optimizer.zero_grad()
    out = model(input=input)[0]

    loss = criterion(out, target)
    loss.backward()
    optimizer.step()

    return [model]

In [ ]:
train.tag("train")
train.send(duet, pointable=True)

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Now STOP and run the Data Scientist notebook until the same checkpoint.

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 2 : Well done!